In [1]:
import tensorflow as tf

# 표준편차를 0.5로 하는 정규분포로 초기화된 텐서를 생성하는 연산자.
weights = tf.Variable(tf.random_normal([300,200], stddev=0.5), name="weights")

# 가중치가 학습될수 없다면 trainable 옵션을 준다.
weights = tf.Variable(tf.random_normal([300,200], stddev=0.5), name="weights"
                     ,trainable=False)

In [2]:
# tf.random_normal 외에도 텐서플로 변수를 초기화하는 몇 가지 방법이 있다.

shape = [2,3]

tf.zeros(shape, dtype=tf.float32, name=None)
tf.ones(shape, dtype=tf.float32, name=None)
tf.random_normal(shape, mean=0.0, stddev=1.0,
                dtype=tf.float32, seed=None, name = None)
tf.truncated_normal(shape, mean=0.0, stddev=1.0,
                   dtype=tf.float32, seed=None, name=None)
tf.random_uniform(shape, minval=0, maxval=None,
                 dtype=tf.float32, seed=None, name=None)

<tf.Tensor 'random_uniform:0' shape=(2, 3) dtype=float32>

In [4]:
# 입력을 딥 모델로 전달하는가? 한 개의 변수는 한 번만 초기화되는 것을 의미하기 떄문에 충분하지않다.

x = tf.placeholder(tf.float32, name="x", shape=[None, 784])
W = tf.Variable(tf.random_uniform([784,10],-1, 1), name="W")
multiply = tf.matmul(x,W)

# x s는 float32에 저장된 데이터의 미니배치를 나타낸다. 784개 열을 가진다는 것을 알수있다.

In [6]:
import tensorflow as tf
from read_data import get_minibatch()

x = tf.placeholder(tf.float32, name="x", shape=[None, 784])
W = tf.Variable(tf.random_uniform([784,10],-1,1), name="W")
b = tf.Variable(tf.zeros([10]), name="biases")
output = tf.matmul(x, W) + b

init_op = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init_op)
feed_dict = {"x" : get_minibatch()}
sess.run(output, feed_dict=feed_dict)

SyntaxError: invalid syntax (<ipython-input-6-6b4dbff2729e>, line 2)

In [11]:
# 이 신경망 설정은 세 개 층을 설명하는 여섯 개의 변수로 구성된다. 
# 이 신경망을 여러 번 사용하려고 하면, my_network처럼 여러 번 호출할 수 있는 작은 함수로 캡슐화하는것이 좋다.

def my_network(input):
    W_1 = tf.Variable(tf.random_uniform([784,100],-1,1), name="W_1")
    b_1 = tf.Variable(tf.zeros([100]), name="biases_1")
    output_1 = tf.matmul(input, W_1) + b_1
    
    W_2 = tf.Variable(tf.random_uniform([100,50],-1,1), name="W_2")
    b_2 = tf.Variable(tf.zeros([50]), name="biases_2")
    output_2 = tf.matmul(output_1, W_2) + b_2
    
    W_3 = tf.Variable(tf.random_uniform([50,10],-1,1), name="W_3")
    b_3 = tf.Variable(tf.zeros([10]), name="biases_3")
    output_3 = tf.matmul(output_2, W_3) + b_3
    
    # 이름 출력
    print("Printing names of weight parameters")
    print(W_1.name, W_2.name, W_3.name)
    print("Printing names of bias parameters")
    print(b_1.name, b_2.name, b_3.name)
    
    return output_3

weights = tf.Variable(tf.random_normal([1000,784], stddev=0.5), name="weights")

my_network(weights)

# 하지만 이렇게 하면 my_network의 두 번째 호출은 첫 번째 호출과 동일한 변수를 사용하지 않는다.
# 대신 변수들의 두 번째 집합을 만들었다. 대부분 사본이 아닌 이 모델들을 재사용 하길 원하는데 이 경우 tf.Variable을 사용해서는 안된다.

Printing names of weight parameters
W_1_3:0 W_2_1:0 W_3:0
Printing names of bias parameters
biases_1_2:0 biases_2_1:0 biases_3:0


<tf.Tensor 'add_3:0' shape=(1000, 10) dtype=float32>

* **tf.get_varialbe([name], [shape], [initializer])**
    
    이 이름을 가진 변수가 존재하는지 확인한다. 변수가 존재한다면 검색하고, 변수가 없으면 shape와 initializer로 생성한다.

* **tf.variable_scope([scope_name])**
    
    네임 스페이스를 관리하고 tf.get_variable이 작동하는 범위를 결정한다.

In [12]:
def layer(input, weight_shape, bias_shape):
    weight_init = tf.random_uniform_initializer(minval=-1, maxval=1)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    b = tf.get_variable("b", vias_shape, initializer=bias_init)
    return tf.matmul(input, W) + b

def my_network(input):
    with tf.variable_scope("layer_1"):
        output_1 = layer(input, [784,100], [100])
        
    with tf.variable_scope("layer_2"):
        output_1 = layer(input, [100,50], [50])
        
    with tf.variable_scope("layer_3"):
        output_1 = layer(input, [50,10], [10])
        
    return output_3

# 이케 하면 주어진 이름의 변수가 인스턴스화되지 않는다.